In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import os as os
import re, string, unicodedata
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nigel.hussain/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nigel.hussain/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
os.getcwd()

'/Users/nigel.hussain/Python Machine Learning API'

In [23]:
# Read CSV
tech_df = pd.read_csv('tech_test_data.csv')
tech_df

# Drop agent rows
tech_customer_df = tech_df[tech_df.message_source == 'customer']
tech_customer_df

# Keep account and order only
tech_info_df = tech_customer_df[tech_customer_df.message_number != 1]
tech_info_df

# Remove non-order messages
tech_info_modified = tech_info_df.drop(19)

tech_info_modified = tech_info_modified.drop(62)

tech_info_modified

,message,case_type,conversation_id,message_id,message_number,message_source
2,"Sure, my order ID is A8B9V1E9 and account numb...",cancel_order,1,3,3,customer
6,"Yeah account number 09832453, order BSD932X0",cancel_order,2,7,3,customer
10,"No worries, my order ID is BEDSW912, let me ch...",cancel_order,3,11,3,customer
11,Account number 67223023,cancel_order,3,12,4,customer
15,Order ID 87GHE8EU and account number 98234321,cancel_order,4,16,3,customer
20,account number 01928340,cancel_order,5,21,4,customer
21,order ID 09BA JH01,cancel_order,5,22,5,customer
25,"account number is 72934923, and order id is 98...",cancel_order,6,26,3,customer
29,Order ID 987yh512 and account number 32430984,cancel_order,7,30,3,customer
33,"account number 85430982, order ID nbreg923",cancel_order,8,34,3,customer


In [27]:
X = tech_info_modified.message

In [28]:
X_list = list(X)
# print(X_list)

X_preprocessed = []
for string in X_list:
    d = replace_contractions(string)
    X_preprocessed.append(d)

print(X_preprocessed)

['Sure, my order ID is A8B9V1E9 and account number is 87630823', 'Yeah account number 09832453, order BSD932X0', 'No worries, my order ID is BEDSW912, let me check the account number', 'Account number 67223023', 'Order ID 87GHE8EU and account number 98234321', 'account number 01928340', 'order ID 09BA JH01', 'account number is 72934923, and order id is 987BGF12', 'Order ID 987yh512 and account number 32430984', 'account number 85430982, order ID nbreg923', 'yeah AN 09823081 order 8n3f0238', 'Order ID NB0293RJ and account number 09128342', 'account number 85430982, order ID nbreg923', 'account number 01928340', 'order ID 09BA JH01', 'Yeah account number 09832453, order BSD932X0', 'Order ID 987yh512 and account number 32430984', 'account number 01928340', 'order ID 09BA JH01', 'Sure, my order ID is A8B9V1E9 and account number is 87630823', 'Of course, order ID 987yh512 and account number 32430984', 'Order ID NB0293RJ and account number 09128342', 'order ID 09BA JH01 and account 09834532'

In [29]:
# First round of preprocessing the training data
def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)

In [30]:
# Tokenise the data

X_tokenised = []
for string in X_preprocessed:
    words = nltk.word_tokenize(string)
    X_tokenised.append(words)

print(X_tokenised)

[['Sure', ',', 'my', 'order', 'ID', 'is', 'A8B9V1E9', 'and', 'account', 'number', 'is', '87630823'], ['Yeah', 'account', 'number', '09832453', ',', 'order', 'BSD932X0'], ['No', 'worries', ',', 'my', 'order', 'ID', 'is', 'BEDSW912', ',', 'let', 'me', 'check', 'the', 'account', 'number'], ['Account', 'number', '67223023'], ['Order', 'ID', '87GHE8EU', 'and', 'account', 'number', '98234321'], ['account', 'number', '01928340'], ['order', 'ID', '09BA', 'JH01'], ['account', 'number', 'is', '72934923', ',', 'and', 'order', 'id', 'is', '987BGF12'], ['Order', 'ID', '987yh512', 'and', 'account', 'number', '32430984'], ['account', 'number', '85430982', ',', 'order', 'ID', 'nbreg923'], ['yeah', 'AN', '09823081', 'order', '8n3f0238'], ['Order', 'ID', 'NB0293RJ', 'and', 'account', 'number', '09128342'], ['account', 'number', '85430982', ',', 'order', 'ID', 'nbreg923'], ['account', 'number', '01928340'], ['order', 'ID', '09BA', 'JH01'], ['Yeah', 'account', 'number', '09832453', ',', 'order', 'BSD932X0

In [31]:
# additional pre-processing
def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def normalize(words):
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    return words

# for later use
def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

In [32]:
X_normalised = []
# print(X_tokenised)
for tokens in X_tokenised:
    words = normalize(tokens)
    X_normalised.append(words)
    
print(X_normalised)

[['Sure', 'order', 'ID', 'A8B9V1E9', 'account', 'number', '87630823'], ['Yeah', 'account', 'number', '09832453', 'order', 'BSD932X0'], ['No', 'worries', 'order', 'ID', 'BEDSW912', 'let', 'check', 'account', 'number'], ['Account', 'number', '67223023'], ['Order', 'ID', '87GHE8EU', 'account', 'number', '98234321'], ['account', 'number', '01928340'], ['order', 'ID', '09BA', 'JH01'], ['account', 'number', '72934923', 'order', 'id', '987BGF12'], ['Order', 'ID', '987yh512', 'account', 'number', '32430984'], ['account', 'number', '85430982', 'order', 'ID', 'nbreg923'], ['yeah', 'AN', '09823081', 'order', '8n3f0238'], ['Order', 'ID', 'NB0293RJ', 'account', 'number', '09128342'], ['account', 'number', '85430982', 'order', 'ID', 'nbreg923'], ['account', 'number', '01928340'], ['order', 'ID', '09BA', 'JH01'], ['Yeah', 'account', 'number', '09832453', 'order', 'BSD932X0'], ['Order', 'ID', '987yh512', 'account', 'number', '32430984'], ['account', 'number', '01928340'], ['order', 'ID', '09BA', 'JH01

In [73]:
# return account number:

AN = []

for string in X_normalised:
#     print(string)
    for token in string:
        if token.isdigit():
            AN.append(token)

print(AN)

['87630823', '09832453', '67223023', '98234321', '01928340', '72934923', '32430984', '85430982', '09823081', '09128342', '85430982', '01928340', '09832453', '32430984', '01928340', '87630823', '32430984', '09128342', '09834532', '67223023']


In [81]:
# Additional preprocessing

for string in X_normalised:
#     print(string)
    for token in string:
        if token == 'id':
            print(string.index('id'))
            print(token)

4
id


In [75]:
X_joined = []

for tokens in X_normalised:
    joined = " ".join(tokens)
    X_joined.append(joined)

print(X_joined)

['Sure order ID A8B9V1E9 account number 87630823', 'Yeah account number 09832453 order BSD932X0', 'No worries order ID BEDSW912 let check account number', 'Account number 67223023', 'Order ID 87GHE8EU account number 98234321', 'account number 01928340', 'order ID 09BA JH01', 'account number 72934923 order id 987BGF12', 'Order ID 987yh512 account number 32430984', 'account number 85430982 order ID nbreg923', 'yeah AN 09823081 order 8n3f0238', 'Order ID NB0293RJ account number 09128342', 'account number 85430982 order ID nbreg923', 'account number 01928340', 'order ID 09BA JH01', 'Yeah account number 09832453 order BSD932X0', 'Order ID 987yh512 account number 32430984', 'account number 01928340', 'order ID 09BA JH01', 'Sure order ID A8B9V1E9 account number 87630823', 'Of course order ID 987yh512 account number 32430984', 'Order ID NB0293RJ account number 09128342', 'order ID 09BA JH01 account 09834532', 'No worries order ID BEDSW912 let check account number', 'Account number 67223023']


In [111]:
ID = []

for string in X_joined:
    if 'order' in string:
        print(string.split('order', 3)[1])
        ID.append(string.split('order',1)[1])
            

print(ID)

 ID A8B9V1E9 account number 87630823
 BSD932X0
 ID BEDSW912 let check account number
 ID 09BA JH01
 id 987BGF12
 ID nbreg923
 8n3f0238
 ID nbreg923
 ID 09BA JH01
 BSD932X0
 ID 09BA JH01
 ID A8B9V1E9 account number 87630823
 ID 987yh512 account number 32430984
 ID 09BA JH01 account 09834532
 ID BEDSW912 let check account number
[' ID A8B9V1E9 account number 87630823', ' BSD932X0', ' ID BEDSW912 let check account number', ' ID 09BA JH01', ' id 987BGF12', ' ID nbreg923', ' 8n3f0238', ' ID nbreg923', ' ID 09BA JH01', ' BSD932X0', ' ID 09BA JH01', ' ID A8B9V1E9 account number 87630823', ' ID 987yh512 account number 32430984', ' ID 09BA JH01 account 09834532', ' ID BEDSW912 let check account number']


In [127]:
# Final ID numbers

ID_tokenised = []
for string in ID:
    words = nltk.word_tokenize(string)
    ID_tokenised.append(words)

print(ID_tokenised)

# ID_final = []


# ID[0]

# for i, strings in enumerate(ID):
#     strings.split()
#     print(i, strings)
#     if len(ID[i]) == 1:
#         ID_final.append(stringss
#     else:
#         ID_final.append(ID[i])

# print(ID_final)

[['ID', 'A8B9V1E9', 'account', 'number', '87630823'], ['BSD932X0'], ['ID', 'BEDSW912', 'let', 'check', 'account', 'number'], ['ID', '09BA', 'JH01'], ['id', '987BGF12'], ['ID', 'nbreg923'], ['8n3f0238'], ['ID', 'nbreg923'], ['ID', '09BA', 'JH01'], ['BSD932X0'], ['ID', '09BA', 'JH01'], ['ID', 'A8B9V1E9', 'account', 'number', '87630823'], ['ID', '987yh512', 'account', 'number', '32430984'], ['ID', '09BA', 'JH01', 'account', '09834532'], ['ID', 'BEDSW912', 'let', 'check', 'account', 'number']]


In [131]:
ID_final = []

for i, strings in enumerate(ID_tokenised):
#     print(i, strings[1])
    if len(ID_tokenised[i]) == 1:
        ID_final.append(ID_tokenised[i][0])
    elif len(ID_tokenised[i])>= 1:
        ID_final.append(ID_tokenised[i][1])

print(ID_final)

['A8B9V1E9', 'BSD932X0', 'BEDSW912', '09BA', '987BGF12', 'nbreg923', '8n3f0238', 'nbreg923', '09BA', 'BSD932X0', '09BA', 'A8B9V1E9', '987yh512', '09BA', 'BEDSW912']


In [ ]:
# finalising 

In [ ]:
# return Order ID

# Capitalise ID


In [72]:
# s = X_normalised[0]

# print(s)

# s[6].isdigit()

# for word in s:
#     print(word.isdigit())

In [ ]:
# return order ID:

In [68]:
# X_joined = []

# for tokens in X_normalised:
#     joined = " ".join(tokens)
#     X_joined.append(joined)

# print(X_joined)

In [69]:
# sample = X_joined[0]
# print(sample)

In [70]:
# return order ID:

# split_string = []
# for string in X_joined:

In [71]:
# my_string="hello python world , i'm a beginner "
# print(my_string.split("world",1))

In [32]:
def oddNumbers(l, r):
    for x in range(1,r):
        if x % 2 != 0:
            print(x)

In [33]:
oddNumbers(1,6)

1
3
5


In [27]:
for i in range(1,5):
    print(i)

1
2
3
4


In [28]:
x = [y for y in range(1,5) if y % 2 !=0]

In [29]:
print(x)

[1, 3]
